In [ ]:
import pandas as pd
import re
import requests
from google.colab import files

In [ ]:
subscriberDF = pd.read_csv('subscriberDF.csv')
subscriberDF

In [ ]:
#function to get rid of 0s and add decimal
def transform_value(value):
    if pd.isnull(value):
        return value
    value_str = str(int(value))
    if value_str.endswith('0000'):
        value_int = int(value / 10000)
        return float(value_int / 100)
    else:
        return value

In [ ]:
#apply to DF
subscriberDF['Purchase Amount'] = subscriberDF['Purchase Amount'].apply(transform_value)
subscriberDF

In [ ]:
#fetch exchange rates
def fetch_exchange_rates(api_key, currencies):
    currency_list = ','.join(currencies)
    url = f'https://openexchangerates.org/api/latest.json?app_id={api_key}&symbols={currency_list}'
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()['rates']
    else:
        raise ValueError(f"Failed to fetch exchange rates. Status code: {response.status_code}")

api_key = '418dd91a76a743c7bee44a1feb77fe51'

currencies = [
    'AED', 'AUD', 'BGN', 'BRL', 'CAD', 'CHF', 'CLP', 'COP', 'CRC', 'CZK',
    'DKK', 'EGP', 'EUR', 'GBP', 'GHS', 'HKD', 'HUF', 'IDR', 'ILS', 'INR',
    'JPY', 'KRW', 'KZT', 'LBP', 'MXN', 'MYR', 'NOK', 'NZD', 'PEN', 'PHP',
    'PLN', 'QAR', 'RON', 'RSD', 'RUB', 'SAR', 'SEK', 'SGD', 'THB', 'TRY',
    'UAH', 'VND', 'ZAR'
]

exchange_rates = fetch_exchange_rates(api_key, currencies)
print(exchange_rates)

In [ ]:
#convert to USD
def convert_to_usd(amount, currency, exchange_rates):
    if currency in exchange_rates:
        return amount / exchange_rates[currency]
    else:
        return amount

def convert_currencies_to_usd(data_frame, amount_column, currency_column, exchange_rates):
    data_frame[amount_column] = data_frame.apply(lambda row: convert_to_usd(row[amount_column], row[currency_column], exchange_rates), axis=1)
    return data_frame

In [ ]:
subscriberDF = convert_currencies_to_usd(subscriberDF, 'Purchase Amount', 'Currency', exchange_rates)
subscriberDF

In [ ]:
appDF = pd.read_csv('appDF.csv')
appDF

In [ ]:
merged_df = appDF.merge(subscriberDF, left_on='Row Labels', right_on='ID')
merged_df

In [ ]:

merged_df.to_csv("merged.csv")
files.download("merged.csv")